In [456]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from plotnine import *

In [457]:
df_draft_raw = pd.read_csv("../data/draft.csv", index_col=0)

In [458]:
df_draft = (df_draft_raw.rename(columns={'Lg': 'league', 'Rd': 'round', 
                                         'Pk': 'pick', 'Tm': 'team', 'Pos': 'position'})
              .assign(Player = lambda x: x.Player.str.split('\\').str.get(0)))
df_draft.columns = df_draft.columns.str.lower()

In [459]:
df_draft.head()

year league  round  pick team          player     age position born  \
Rk                                                                        
1   2016    NBA      1     1  PHI     Ben Simmons  19.343      G-F   au   
2   2016    NBA      1     2  LAL  Brandon Ingram  18.299        F   us   
3   2016    NBA      1     3  BOS    Jaylen Brown  19.247      F-G   us   
4   2016    NBA      1     4  PHO   Dragan Bender  18.223        F   ba   
5   2016    NBA      1     5  MIN       Kris Dunn  22.103        G   us   

                       college  
Rk                              
1   Louisiana State University  
2              Duke University  
3     University of California  
4                          NaN  
5           Providence College

In [460]:
df_nba_raw = pd.read_csv("../data/nbarookiedata.csv", skiprows=1, index_col=0)

In [461]:
df_nba = (df_nba_raw
             .assign(Player = lambda x: x.Player.str.split('\\').str.get(0),
                     Season = lambda x: pd.to_numeric(x.Season.str.split('-').str.get(0)))
             [['Player', 'Tm', 'Season', 'PTS', 'TRB', 'AST', 'STL', 'BLK']]
             .rename(columns={'Player': 'player', 'Tm': 'team', 'Season': 'year', 
                              'PTS': 'point', 'TRB': 'rebound', 'AST': 'assist',
                              'STL': 'steal', 'BLK': 'block'}))

In [462]:
df_nba.head()

player team  year  point  rebound  assist  steal  block
Rk                                                                  
1      Alex Abrines  OKC  2016    6.0      1.3     0.6    0.5    0.1
2         Ron Baker  NYK  2016    4.1      1.9     2.1    0.7    0.2
3      Wade Baldwin  MEM  2016    3.2      1.4     1.8    0.5    0.2
4     Malik Beasley  DEN  2016    3.8      0.8     0.5    0.3    0.0
5   DeAndre' Bembry  ATL  2016    2.7      1.6     0.7    0.2    0.1

In [463]:
df_final = pd.merge(df_draft, df_nba, how='left', on=['player', 'team', 'year'])

In [464]:
# There are a lot of players drafted but don't appear in the NBA dataset, why?
df_final.head()

year league  round  pick team          player     age position born  \
0  2016    NBA      1     1  PHI     Ben Simmons  19.343      G-F   au   
1  2016    NBA      1     2  LAL  Brandon Ingram  18.299        F   us   
2  2016    NBA      1     3  BOS    Jaylen Brown  19.247      F-G   us   
3  2016    NBA      1     4  PHO   Dragan Bender  18.223        F   ba   
4  2016    NBA      1     5  MIN       Kris Dunn  22.103        G   us   

                      college  point  rebound  assist  steal  block  
0  Louisiana State University    NaN      NaN     NaN    NaN    NaN  
1             Duke University    9.4      4.0     2.1    0.6    0.5  
2    University of California    6.6      2.8     0.8    0.4    0.2  
3                         NaN    3.4      2.4     0.5    0.2    0.5  
4          Providence College    3.8      2.1     2.4    1.0    0.5

In [465]:
df_ncaa_raw = pd.read_csv('../data/ncaa2.csv', index_col=0)

In [466]:
df_ncaa_raw.columns

Index(['Name', 'Team', 'GP', 'Min', 'Pts', 'FG', 'FGA', 'FG%', '2Pt', '2PtA',
       '2P%', '3Pt', '3PtA', '3P%', 'FTM', 'FTA', 'FT%', 'Off', 'Def', 'TOT',
       'Asts', 'Stls', 'Blks', 'TOs', 'PFs', 'year', 'PTs/g', 'FGA/g',
       'Pts/Play', 'TS%', 'eFG%', 'FTA/FGA', '3PA/FGA', 'Ast/g', 'Ast/FGA',
       'A/TO', 'PPR', 'BK/g', 'STL/g', 'PF/g'],
      dtype='object')

In [467]:
df_ncaa = (df_ncaa_raw
              [['Name', 'Team', 'year', 'Pts', 'TOT', 'Asts', 'Stls', 'Blks']]
              .rename(columns={'Name': 'player', 'Team': 'college', 'year': 'col_year',
                               'Pts': 'col_point', 'TOT': 'col_rebound', 'Asts': 'col_assist',
                               'Stls': 'col_steal', 'Blks': 'col_block'}))

In [468]:
df_ncaa.head()

player           college  col_year  col_point  col_rebound  \
0       Luis Rivas  Western Illinois      2002       27.9         10.7   
1   Steve Reynolds  Western Michigan      2002       27.8          6.5   
2   Michael Watson              UMKC      2002       24.9          3.6   
3     Tyrone Hayes             Idaho      2002       34.3         12.3   
4  Travis Robinson      Jacksonville      2002       27.4          7.5   

   col_assist  col_steal  col_block  
0         0.0        0.0        0.5  
1         2.0        1.2        0.1  
2         3.2        2.1        0.2  
3         0.0        2.5        2.5  
4         1.1        1.9        0.3

In [469]:
df_final = pd.merge(df_final, df_ncaa, how='inner', on=['player'])

In [470]:
df_draft.shape

(1125, 10)

In [471]:
df_final.shape

(1544, 22)

In [472]:
print("Number of unique players: {}".format(len(pd.unique(df_final.player))))

Number of unique players: 613


In [473]:
# Only keep the last year of college performance
df_final = (df_final.groupby('player')
        .apply(lambda df: df.sort_values(by='col_year', ascending=False).iloc[0, :]))

In [474]:
df_final

year league  round  pick team             player     age  \
player                                                                        
A.J. Hammons       2016    NBA      2    46  DAL       A.J. Hammons  23.306   
A.J. Price         2009    NBA      2    52  IND         A.J. Price  22.267   
Aaron Brooks       2007    NBA      1    26  HOU       Aaron Brooks  22.170   
Aaron Gordon       2014    NBA      1     4  ORL       Aaron Gordon  18.288   
Aaron Gray         2007    NBA      2    49  CHI         Aaron Gray  22.208   
Aaron White        2015    NBA      2    49  WAS        Aaron White  22.293   
Acie Law           2007    NBA      1    11  ATL           Acie Law  22.159   
Adam Haluska       2007    NBA      2    43  NOK       Adam Haluska     NaN   
Adam Morrison      2006    NBA      1     3  CHA      Adam Morrison  21.349   
Adreian Payne      2014    NBA      1    15  ATL      Adreian Payne  23.133   
Ahmad Nivins       2009    NBA      2    56  DAL       Ahmad Nivins     NaN   
Al Horford         2007    NBA      1     3  ATL         Al Horford  21.030   
Al Thornton        2007    NBA      1    14  LAC        Al Thornton  23.209   
Al-Farouq Aminu    2010    NBA      1     8  LAC    Al-Farouq Aminu  19.281   
Alando Tucker      2007    NBA      1    29  PHO      Alando Tucker  23.143   
Alec Brown         2014    NBA      2    50  PHO         Alec Brown  21.343   
Alec Burks         2011    NBA      1    12  UTA         Alec Burks  19.343   
Alex Acker         2005    NBA      2    60  DET         Alex Acker  22.164   
Alex Len           2013    NBA      1     5  PHO           Alex Len  20.016   
Alex Oriakhi       2013    NBA      2    57  PHO       Alex Oriakhi  23.012   
Alexander Johnson  2006    NBA      2    45  IND  Alexander Johnson  23.146   
Allen Crabbe       2013    NBA      2    31  CLE       Allen Crabbe  21.084   
Amir Johnson       2005    NBA      2    56  DET       Amir Johnson  18.063   
Andre Drummond     2012    NBA      1     9  DET     Andre Drummond  18.327   
Andre Emmett       2004    NBA      2    35  SEA       Andre Emmett  21.307   
Andre Iguodala     2004    NBA      1     9  PHI     Andre Iguodala  20.153   
Andre Roberson     2013    NBA      1    26  MIN     Andre Roberson  21.211   
Andrew Bogut       2005    NBA      1     1  MIL       Andrew Bogut  20.217   
Andrew Goudelock   2011    NBA      2    46  LAL   Andrew Goudelock  22.203   
Andrew Harrison    2015    NBA      2    44  PHO    Andrew Harrison  20.245   
...                 ...    ...    ...   ...  ...                ...     ...   
Trey Burke         2013    NBA      1     9  MIN         Trey Burke  20.232   
Trey Thompkins     2011    NBA      2    37  LAC     Trey Thompkins  21.039   
Tristan Thompson   2011    NBA      1     4  CLE   Tristan Thompson  20.107   
Troy Bell          2003    NBA      1    16  BOS          Troy Bell  22.233   
Ty Lawson          2009    NBA      1    18  MIN          Ty Lawson  21.240   
Tyler Ennis        2014    NBA      1    18  PHO        Tyler Ennis  19.311   
Tyler Hansbrough   2009    NBA      1    13  IND   Tyler Hansbrough  23.240   
Tyler Harvey       2015    NBA      2    51  ORL       Tyler Harvey  21.348   
Tyler Honeycutt    2011    NBA      2    35  SAC    Tyler Honeycutt  20.348   
Tyler Zeller       2012    NBA      1    17  DAL       Tyler Zeller  22.168   
Tyreke Evans       2009    NBA      1     4  SAC       Tyreke Evans  19.284   
Tyrone Wallace     2016    NBA      2    60  UTA     Tyrone Wallace  22.019   
Tyrus Thomas       2006    NBA      1     4  POR       Tyrus Thomas  19.320   
Tyshawn Taylor     2012    NBA      2    41  POR     Tyshawn Taylor  22.083   
Vernon Macklin     2011    NBA      2    52  DET     Vernon Macklin  24.282   
Victor Oladipo     2013    NBA      1     2  ORL     Victor Oladipo  21.059   
Vincent Yarbrough  2002    NBA      2    33  DEN  Vincent Yarbrough  21.102   
Von Wafer          2005    NBA      2    39  LAL          Von Wafer  

# Model building

In [475]:
df_final = df_final.dropna()
X = df_final[['round', 'pick', 'position',
              'col_point', 'col_rebound', 'col_assist', 'col_steal', 'col_block']]
X = pd.get_dummies(X, columns=['position'])
y = df_final['point']

In [476]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [477]:
rf = RandomForestRegressor(criterion='mse', n_jobs=-1)
rf.get_params()

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
gs = GridSearchCV(estimator=rf, 
                  param_grid=[{'n_estimators': [100, 1000, 10000],
                               'min_samples_leaf': [1, 10, 100]}],
                  cv=10, scoring='neg_mean_squared_error')

In [ ]:
gs.fit(X_train, y_train)

In [ ]:
gs.best_params_

In [ ]:
best_rf = gs.best_estimator_
best_rf.fit(X_train, y_train)

In [ ]:
mean_squared_error(y_train, best_rf.predict(X_train))

In [ ]:
y_pred = best_rf.predict(X_test)

In [ ]:
print("Min / Mean / Max of Point: {} {} {}".format(np.min(df_final.point),
                                                   np.mean(df_final.point),
                                                   np.max(df_final.point)))
print("Train MSE: {.3f}; Test MSE: {.3f}".format(mean_squared_error(y_train, best_rf.predict(X_train),
                                                 mean_squared_error(y_test, best_rf.predict())))

In [455]:
ggplot(data=df_final) + geom_histogram(aes(x='point'))

/Users/anh/miniconda3/lib/python3.6/site-packages/plotnine/stats/stat_bin.py:90: UserWarning: 'stat_bin()' using 'bins = 15'. Pick better value with 'binwidth'.
  warn(msg.format(params['bins']))


<ggplot: (-9223372036549853745)>